# Naive Forecasts

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

In [3]:
%aimport src.custom_estimators
from src.custom_estimators import DFColumnRenamer, MultiTSCustomNaiveRegressor

%aimport src.metrics_helpers
from src.metrics_helpers import score_predictions

%aimport src.utils
from src.utils import show_df

<a id="table-of-contents"></a>

## [Table of Contents](#table-of-contents)

0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load Data](#load-data)
3. [Split Data](#split-data)
4. [Make and Score Naive Forecasts](#make-and-score-naive-forecasts)

<a id="about"></a>

## 0. [About](#about)

When working with timeseries data, it is important to assess the performance of basic or naive forecasting methods prior to developing more sophisticated methods. Often, such approaches can provide a reliable lower bound on model performance against which the performance of a sophisticated forecasting technique can be assessed.

The naive strategy we will use here is to take the average of the electricity usage during the same period as the holdout data but from previous years. We'll use the years from 2016 to 2019 inclusive. We will naively assume that this average consumption is the forecasted load for the holdout period. Finally, we'll score this naive forecast against the true values from the holdout data.

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

In [4]:
PROJ_ROOT_DIR = os.getcwd()

In [5]:
index_name = "utc_timestamp"

train_val_start = "2015-01-01 00:00:00"
train_val_end = "2020-07-01 23:00:00"
test_start = "2020-07-02 00:00:00"
test_end = "2020-09-30 23:00:00"

naive_cutoffs = [
    ["2016-06-30 00:00:00", "2016-09-28 23:00:00"],
    ["2017-06-29 00:00:00", "2017-09-27 23:00:00"],
    ["2018-07-05 00:00:00", "2018-10-03 23:00:00"],
    ["2019-07-04 00:00:00", "2019-10-02 23:00:00"],
]

renamer = {index_name: "ds", "load": "y"}

In [6]:
def get_metrics(df):
    df = df.dropna()
    return pd.DataFrame.from_dict(
        score_predictions(df["y"], df["yhat"], get_r2=True), orient="index"
    ).T

In [ ]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
processed_data_dir = os.path.join(data_dir, "processed")

processed_data_filepath = glob(os.path.join(processed_data_dir, "*.parquet.gzip"))[-1]
print(processed_data_filepath)

<a id="load-data"></a>

## 2. [Load Data](#load-data)

In [8]:
%%time
df = pd.read_parquet(processed_data_filepath)
show_df(df[["country", index_name, "load"]], 1)
display(df["country"].value_counts().to_frame().T)

,country,utc_timestamp,load
0,BE,2015-01-01 00:00:00,9.484000
503999,PL,2020-09-30 23:00:00,nan


,BE,CH,CZ,DE,ES,FR,HR,IT,NL,PL
country,50400,50400,50400,50400,50400,50400,50400,50400,50400,50400


CPU times: user 475 ms, sys: 97.7 ms, total: 572 ms
Wall time: 273 ms


<a id="split-data"></a>

## 3. [Split Data](#split-data)

We'll now create the following two data splits
- training
- testing

In [9]:
df_train_val_naive = df[
    (df[index_name] >= train_val_start) & (df[index_name] <= train_val_end)
].reset_index(drop=True)[["country", index_name, "load"]]
df_test_naive = df[
    (df[index_name] >= test_start) & (df[index_name] <= test_end)
].reset_index(drop=True)[["country", index_name, "load"]]
print(len(df_train_val_naive), len(df_test_naive))

482160 21840


<a id="make-and-score-naive-forecasts"></a>

## 4. [Make and Score Naive Forecasts](#make-and-score-naive-forecasts)

We'll now make the naive forecasts using the average of electricity consumption by country upto the specified cutoff dates

In [10]:
!cat -n src/custom_estimators.py | sed -n -e 1,10p -e 86,198p

     1	#!/usr/bin/env python3
     2	# -*- coding: utf-8 -*-
     3	
     4	
     5	import numpy as np
     6	import pandas as pd
     7	from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
     8	from sklearn.utils.validation import check_array, check_X_y
     9	
    10	
    86	class MultiTSCustomNaiveRegressor(BaseEstimator, RegressorMixin):
    87	    """
    88	    Notes
    89	    -----
    90	    1. The length of the datetimes covered by the start and end of each
    91	       naive cutoff pair must match the unique dates in the
    92	       - forecast horizon
    93	       - out-of-sample data, on which .predict() will be called
    94	    2. .predict() can be called on the out-of-sample data only
    95	
    96	    Usage
    97	    -----
    98	    # Inputs
    99	    train_val_start = "1800-01-01"
   100	    train_val_end = "1802-09-30"
   101	    test_start = "1802-10-01"
   102	    test_end = "1802-12-31"
   103	    naive_cutoffs = [
   104	        ["180

In [11]:
est = Pipeline(
    [
        ("rename", DFColumnRenamer(renamer)),
        ("reg", MultiTSCustomNaiveRegressor(naive_cutoffs, "ds", "country")),
    ]
)
est.fit(df_train_val_naive)
df_naive_pred = est.predict(df_test_naive)

We'll now score the forecasted values against the true consumption from the test split

In [12]:
dfs_scores_naive = (
    df_naive_pred.merge(
        df_test_naive.rename(columns=renamer), on=["country", "ds"], how="left"
    )
    .groupby("country")
    .apply(get_metrics)
    .reset_index(level=1, drop=True)
)
display(
    dfs_scores_naive.style.set_caption("OOS Evaluation Metrics from Naive Forecast")
)

,rmse,mae,smape(%),mse,type,r2,rmspe(%)
country,,,,,,,
BE,0.602948,0.509517,5.672767,0.363546,pred,0.691537,7.011721
CH,0.495981,0.366660,5.941733,0.245997,pred,0.457985,7.633739
CZ,0.309780,0.235938,3.677630,0.095963,pred,0.914724,5.132611
DE,2.756419,2.345765,4.460283,7.597846,pred,0.905668,5.368848
ES,1.504804,1.224064,4.305789,2.264434,pred,0.877177,5.486631
FR,3.255397,1.918370,4.266561,10.597610,pred,0.724029,5.522784
HR,0.159696,0.124162,6.083965,0.025503,pred,0.803608,8.119956
IT,2.328027,1.895235,5.563552,5.419709,pred,0.881743,6.883235
NL,1.473924,1.133056,9.310599,2.172453,pred,0.012187,13.175371


A non-naive model with skill should perform better than these naive forecast metrics.

---

<span style="float:left">
    <a href="./0_get_data.ipynb"><<< 0 - Get Data</a>
</span>

<span style="float:right">
    <a href="./2_train.ipynb">2 - Prophet model training >>></a>
</span>